In [1]:
# define items.
## names are variants to allow for proper grammar in print statements.
couch = {
    "name": "couch",
    "type": "furniture",
}
piano = {
    "name": "piano",
    "type": "furniture",
}
queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}
double_bed = {
    "name": "double bed",
    "type": "furniture",
}
dresser = {
    "name": "dresser",
    "type": "furniture",
}
dining_table = {
    "name": "dining table",
    "type": "furniture",
}

# define doors.
door_a = {
    "name": "Door A",
    "type": "door",
}
door_b = {
    "name": "Door B",
    "type": "door",
}
door_c = {
    "name": "Door C",
    "type": "door",
}
door_d = {
    "name": "Door D",
    "type": "door",
}

# define keys.
key_a = {
    "name": "the key for Door A",
    "type": "key",
    "target": door_a,
}
key_b = {
    "name": "the key for Door B",
    "type": "key",
    "target": door_b,
}
key_c = {
    "name": "the key for Door C",
    "type": "key",
    "target": door_c,
}
key_d = {
    "name": "the key for Door D",
    "type": "key",
    "target": door_d,
}

# define rooms.
game_room = {
    "name": "the Game Room",
    "type": "room",
}
bedroom_1 = {
    "name": "Bedroom 1",
    "type": "room",
}
bedroom_2 = {
    "name": "Bedroom 2",
    "type": "room",
}
living_room = {
    "name": "Living Room",
    "type": "room",
}
outside = {
  "name": "outside"
}

# define which items/rooms are related

object_relations = {
    "the Game Room": [couch, piano, door_a],
    "Bedroom 1": [queen_bed, door_a, door_b, door_c],
    "Bedroom 2": [double_bed, dresser, door_b],
    "Living Room": [dining_table, door_c, door_d],
    "outside": [door_a],
    "piano": [key_a],
    "queen bed": [key_b],
    "double bed": [key_c],
    "dresser": [key_d],
    "Door A": [game_room, bedroom_1],
    "Door B": [bedroom_1, bedroom_2],
    "Door C": [bedroom_1, living_room],
    "Door D": [living_room, outside]
}

# Define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [2]:
import re

def start_game():
    """
    Start the game. Print the introduction statement.
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!\n\nHint: It may help to begin by exploring the room you are currently in.")
    """
    Run the play_room function on the value for the key current_room from the dictionary game_state.
    """
    play_room(game_state["current_room"])

def play_room(current_room):
    """
    Play a room. First define the <current_room> variable as the value for the key current_room from the dictionary game_state. 
    """
    game_state["current_room"] = current_room
    """
    Then check if <room> is the target room.
    """
    if(current_room == game_state["target_room"]):
        """
        If it is, the game will end with success.
        """
        print("Congrats! You escaped the room!")
    else:
        """
        Otherwise, let player choose to either explore (list all items in this room) or examine an item here.
        """
        print("\nYou are currently in " + current_room["name"] + ".")
        intended_action = input("What would you like to do? Type 'explore' or 'examine': ").strip().lower()
        """
        If the player chooses to explore, run the explore_room function on <current_room> and restart the play_room function on <room>. 
        If the player chooses to examine, run the examine_item function on the player's item choice.
        Otherwise, the input was invalid. Print the error statement and restart the play_room function on <current_room>.
        """
        if intended_action == "explore":
            explore_room(current_room)
            play_room(current_room)
        elif intended_action == "examine":
            item_name = input("What would you like to examine: ").strip().lstrip('the a')
            if re.findall("(?:d|D).*", item_name):
                item_name = item_name.title()
            else:
                item_name = item_name.lower()
            examine_item(item_name)
        else:
            print("Not sure what you mean. Type in either 'explore' or 'examine'.\n")
            play_room(current_room)

def explore_room(current_room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = ["a " + i["name"] for i in object_relations[current_room["name"]] if i["type"] != "door"]
    items = items + [i["name"] for i in object_relations[current_room["name"]] if i["type"] == "door"]
    items[-1] = "and " + items[-1] + "."
    print("You explore " + current_room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    First define the <connected_rooms> variable for the current door being examined as the value for the key Door X from the dictionary game_state.
    """
    connected_rooms = object_relations[door["name"]]
    """
    Return the room that is not the current_room.
    """
    for room in connected_rooms:
        if(current_room != room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if item["name"] == item_name:
            if item["type"] == "door":
                output = "You examine " + item["name"] + ". "
                have_key = False
                for key in game_state["keys_collected"]:
                    if key["target"] == item:
                        have_key = True
                if have_key:
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked, but you don't have the key."
            else:
                output = "You examine the " + item["name"] + ". "
                if item["name"] in object_relations and len(object_relations[item["name"]]) > 0:
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if output == None:
        print("You cannot find that item in the current room.")
        explore_room(current_room)
        examine_item(input("What would you like to examine: ").strip())
    
    if next_room != "" and input("Do you want to go to the next room? Enter 'yes' or 'no': ").strip() == 'yes':
        play_room(next_room)
    else:
        play_room(current_room)

In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!

Hint: It may help to begin by exploring the room you are currently in.

You are currently in the Game Room.
What would you like to do? Type 'explore' or 'examine': examine
What would you like to examine: piAno
You examine the piano. You find the key for Door A.

You are currently in the Game Room.
What would you like to do? Type 'explore' or 'examine': examine
What would you like to examine: the piano
You examine the piano. There isn't anything interesting about it.

You are currently in the Game Room.
What would you like to do? Type 'explore' or 'examine': examine
What would you like to examine: DoOR a
You examine Door A. You unlock it with a key you have.
Do you want to go to the next room? Enter 'yes' or 'no': yes

You are 